In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
import os,sys
sys.path.append('./misc/lib/python3.7/site-packages')

import math
import numpy as np
import requests
import nglview as nv
import ipywidgets as widgets
%matplotlib notebook
import matplotlib.pyplot as plt
from IPython.display import display, display_markdown
from ipywidgets import Layout, HTML
from pathlib import Path

import parmed as pmd
import re

import hublib.use
from hublib.ui import FileUpload, Download
from hublib.cmd import runCommand

from scipy.ndimage import gaussian_filter

from scipy import spatial

import time

%matplotlib notebook
# %use gromacs-2018.4
%use pymol-1.8.4

import hublib.usesh
%usesh gromacs-2018.4

np.set_printoptions(precision=8)
np.set_printoptions(suppress=True)


HTMLButtonPrompt = '''<html>
<head>
<meta name="viewport" content="width=device-width, initial-scale=1">
</head>
<body>
<a href="{link}" target="_blank" >
<button class="p-Widget jupyter-widgets jupyter-button widget-button mod-warning" style="width:150px; background-color:#CCCCCC; font-size:10pt; color:black">{text}</button>
</a>
</body>
</html>
'''   


In [ ]:
def add_hydrogens():
    
    pfile = 'pigments.pdb'
    
    # return message
    mess = ''
    
    # error flag
    error = False
    
    struc = pmd.load_file(pfile)

    hbond_length = 1.0

    # Create an empty structure for the pigment data
    pigstruc = pmd.structure.Structure()

    for res in struc.residues:
        tfile = res.name + '.itp'
        if os.path.exists(tfile)==False:
            mess += 'Could not locate itp file for pigment ' + res.name + '\n'
            error = True
            break
        else:
            # First load atom data
            with open(tfile) as tfd:
                in_atoms = False
                AtNums = []
                AtNames = []
                for line in tfd:
                    if in_atoms and line[0]=='[':
                        in_atoms = False

                    if in_atoms and len(line.split())>0 and line.split()[0].isdigit():
                        AtNums.append(int(line.split()[0]))
                        AtNames.append(line.split()[1])

                    if line[0:9]=='[ atoms ]':
                        in_atoms = True
            tfd.close()        
            AtNums = np.array(AtNums)

            # Now load bond data
            with open(tfile) as tfd:
                in_bonds = False
                Bonds = []
                for line in tfd:
                    if in_bonds and line[0]=='[':
                        in_bonds = False

                    if in_bonds and len(line.split())>0 and line.split()[0].isdigit():
                        dat = [int(line.split()[0]), int(line.split()[1])]
                        Bonds.append(dat)

                    if line[0:9]=='[ bonds ]':
                        in_bonds = True
            tfd.close()
            Bonds = np.array(Bonds)

            Nheavy = 0
            Nhydro = 0
            for name in AtNames:
                if name[0]!='H':
                    Nheavy += 1
                else:
                    Nhydro += 1

            mess += 'Should be ' + str(Nheavy) + ' heavy atoms and ' + str(Nhydro) + ' hydrogens\n'
            mess += 'PDB structure contains ' + str(len(res.atoms)) + ' atoms\n'

            addH = False
            if Nheavy==len(res.atoms):
                mess += 'Assuming PDB file contains no hydrogens\n'
                addH = True
            elif (Nheavy+Nhydro)==len(res.atoms):
                mess += 'Assuming that all hydrogens are included in PDB\n'
                df = struc.to_dataframe()
                pigstruc = pigstruc + struc[df.resnum==res.number]
            else:
                mess += 'Atom mismatch between PDB and itp files. Are all heavy atoms present?\n'
                break
                error = True

            if error==False and addH==True:

                # Generate an empty structure for this pigment
                newstruc = pmd.structure.Structure()

                # Generate an empty coordinates list for atom positions
                coords = []

                # This will count how far we are through the atoms of the original residue
                pdb_ndx = 0
                HeavyParts = []  # Index in parmed struc of heavy-atom partners
                HydroParts = []  # Index in parmed struc of hydrogen partners
                for n in range(0, len(AtNames)):
                    name = AtNames[n]
                    num = AtNums[n]
                    if name[0]!='H':
                        newat = res.atoms[pdb_ndx]
                        newstruc.add_atom(newat, res.name, res.number, chain=res.chain)
                        coords.append(struc.coordinates[struc.atoms.index(newat)])
                        pdb_ndx += 1

                        blist = Bonds[np.where((Bonds[:,0]==num)+(Bonds[:,1]==num))]
                        heavy_partners = []
                        hydro_partners = []
                        for bond in blist:
                            if bond[0]==num:
                                part = bond[1]
                            else:
                                part = bond[0]
                            if AtNames[part-1][0]=='H':
                                hydro_partners.append(part)
                            else:
                                heavy_partners.append(part)

                        # Update bonding partners in partner lists
                        HeavyParts.append(np.array(heavy_partners)-1)
                        HydroParts.append(np.array(hydro_partners)-1)

                        if len(hydro_partners)>0:
                            parent_index = len(HeavyParts)-1
                            for h in range(0, len(hydro_partners)):
                                newat = pmd.topologyobjects.Atom(name='H', atomic_number=1)
                                newstruc.add_atom(newat, res.name, res.number, chain=res.chain)
                                coords.append([0.0, 0.0, 0.0])
                                HeavyParts.append([parent_index])
                                HydroParts.append([])
                newstruc.coordinates = np.array(coords)

                # Now all atoms are added, but H atom coordinates are not set. 
                # We need now to calculate H positions from bonding partners
                coords = newstruc.coordinates.copy()
                for n in range(0, len(AtNames)):
                    at = newstruc.atoms[n]
                    if at.name!='H':
                        Nhydros = len(HydroParts[n])
                        Nheavys = len(HeavyParts[n])


                        # parent atom coordinates
                        xpar = newstruc.coordinates[n]

                        # First calculate average position of bond partners
                        com = np.zeros((3,))
                        for part in HeavyParts[n]:
                            com += newstruc.coordinates[part]
                        com /= Nheavys
                        part_axis = (xpar - com)/np.linalg.norm(xpar - com)

                        # If only one hydrogen, desired bond axis points from 
                        # average position of bonding partners to the parent atom
                        if Nhydros==1:
                            coords[n+1] = xpar + part_axis*hbond_length

                        if Nhydros==2:

                            # SP2 hybridization
                            if (Nhydros+Nheavys)==3:

                                # To define bonding plane, we need a 
                                # heavy-atom bond partner FOR the bond partner

                                # part1 is the first heavy-atom partner of the parent
                                part1 = HeavyParts[n][0]
                                xp1 = newstruc.coordinates[part1]

                                # part2 is the first heavy-atom partner of part1
                                part2 = HeavyParts[part1][0]
                                xp2 = newstruc.coordinates[part2]

                                # xpar, xp1, and xp2 define the bonding plane
                                nvec = np.cross(xpar-xp1, xp2-xp1)
                                nvec /= np.linalg.norm(nvec)

                                # Rotation objects for transforming H-atom positions
                                Rot1 = spatial.transform.Rotation.from_rotvec(nvec*60.0*(np.pi/180.0))
                                Rot2 = spatial.transform.Rotation.from_rotvec(-nvec*60.0*(np.pi/180.0))

                                coords[n+1] = xpar + hbond_length*Rot1.apply(part_axis)
                                coords[n+2] = xpar + hbond_length*Rot2.apply(part_axis)

                            # SP3 hybridization
                            if (Nhydros+Nheavys)==4:

                                # To define bonding plane, we need two heavy-atom
                                # partners for parent atom
                                part1 = HeavyParts[n][0]
                                part2 = HeavyParts[n][1]

                                # xpar, xp1, and xp2 define the bonding plane
                                xp1 = newstruc.coordinates[part1]
                                xp2 = newstruc.coordinates[part2]
                                noop = np.cross(xp1-xpar, xp2-xpar)
                                noop /= np.linalg.norm(nvec)

                                # nvec is orthogonal to the bonding plane of the two hydrogens
                                nvec = np.cross(part_axis, noop)
                                nvec /= np.linalg.norm(nvec)

                                Rot1 = spatial.transform.Rotation.from_rotvec(nvec*0.5*109.5*(np.pi/180.0))
                                Rot2 = spatial.transform.Rotation.from_rotvec(-nvec*0.5*109.5*(np.pi/180.0))

                                coords[n+1] = xpar + hbond_length*Rot1.apply(part_axis)
                                coords[n+2] = xpar + hbond_length*Rot2.apply(part_axis)


                        # Only SP3 hybridization is possible here
                        if Nhydros==3:

                            # part_axis points along the average axis of the 
                            # three hydrogen atoms. We need just to rotate one
                            # hydrogen atom by (180-109.5) degrees (along any 
                            # axis perpendicular to part_axis) and then rotate 
                            # the position of the other two by +/-120 degrees 
                            # around the part_axis (starting from the first H 
                            # position). 

                            # For concreteness, we'll define the initial 
                            # rotation axis using the cross product of 
                            # the parent particle, its first heavy-atom
                            # bond partner, and the partern's first partner.
                            part1 = HeavyParts[n][0]
                            part2 = HeavyParts[part1][0]
                            xp1 = newstruc.coordinates[part1]
                            xp2 = newstruc.coordinates[part2]
                            nvec = np.cross(xpar-xp1, xp2-xp1)
                            nvec /= np.linalg.norm(nvec)

                            # Rot1 rotates the first H-atom position to be 109.5 degrees from part_axis
                            Rot1 = spatial.transform.Rotation.from_rotvec((180.0-109.5)*(np.pi/180.0)*nvec)

                            # dX1 is a unit vector 109.5 degrees from part_axis
                            dX1 = Rot1.apply(part_axis.copy())

                            # Rot2 and Rot3 rotate dX1 by +/-120 degrees around part_axis
                            Rot2 = spatial.transform.Rotation.from_rotvec((120.0)*(np.pi/180.0)*part_axis)
                            Rot3 = spatial.transform.Rotation.from_rotvec((-120.0)*(np.pi/180.0)*part_axis)

                            coords[n+1] = xpar + hbond_length*dX1
                            coords[n+2] = xpar + hbond_length*Rot2.apply(dX1)
                            coords[n+3] = xpar + hbond_length*Rot3.apply(dX1)

                newstruc.coordinates = coords
                pigstruc = pigstruc + newstruc
                mess += "\n"

    if error==False:
        pigstruc.write_pdb('pigmentsh.pdb')
        out = !{"gmx editconf -f pigmentsh.pdb -o pigments.gro"}
    return mess

In [ ]:
print(add_hydrogens())